## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\a2541\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',input_shape=(32, 32, 3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, kernel_size=(3, 3), padding='same'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

classifier.add(Convolution2D(128, kernel_size=(3, 3), padding='same'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

classifier.add(Convolution2D(256, kernel_size=(3, 3), padding='same'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=500,epochs=15,validation_data=(x_test,y_test))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


C:\Users\a2541\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\a2541\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 18s 354us/step - loss: 1.4056 - acc: 0.5279 - val_loss: 1.0833 - val_acc: 0.6256
Epoch 2/15
50000/50000 [==============================] - 15s 291us/step - loss: 0.8593 - acc: 0.7053 - val_loss: 0.8989 - val_acc: 0.6933
Epoch 3/15
50000/50000 [==============================] - 15s 293us/step - loss: 0.6378 - acc: 0.7814 - val_loss: 0.8475 - val_acc: 0.7142
Epoch 4/15
50000/50000 [==============================] - 15s 292us/step - loss: 0.4646 - acc: 0.8449 - val_loss: 0.8381 - val_acc: 0.7275
Epoch 5/15
50000/50000 [==============================] - 15s 292us/step - loss: 0.3284 - acc: 0.8930 - val_loss: 0.8700 - val_acc: 0.7292
Epoch 6/15
50000/50000 [==============================] - 15s 294us/step - loss: 0.1996 - acc: 0.9409 - val_loss: 0.9015 - val_acc: 0.7355
Epoch 7/15
50000/50000 [==============================] - 15s 294us/step - loss: 0.1170 - acc: 0.9701 - val_loss: 0.

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.07876120e-04, 7.32771639e-08, 2.00796116e-04, 9.99305248e-01,
        1.17443895e-04, 2.23026364e-05, 5.70799348e-05, 1.06476975e-04,
        8.27317635e-05, 3.07285225e-10]], dtype=float32)

In [6]:
input_test = np.expand_dims(((x_test[0]-mean_train)/(std_train+1e-7)), axis=0)
classifier.predict(input_test)

array([[9.48378511e-05, 8.26557525e-08, 1.79554030e-04, 9.99367893e-01,
        1.05728664e-04, 2.67019695e-05, 6.04389534e-05, 9.43414416e-05,
        7.04302074e-05, 3.22800120e-10]], dtype=float32)